In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [6]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser, host='/var/run/postgresql')
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [7]:
query = """
SELECT * FROM venttype
"""
vt = pd.read_sql_query(query, con)
vt.head()

,icustay_id,charttime,noninvvent,mechvent,extubated,selfextubated
0,200003.0,2199-08-02 19:15:00,0,0,0,0
1,200003.0,2199-08-02 20:00:00,0,0,0,0
2,200003.0,2199-08-02 21:00:00,0,0,0,0
3,200003.0,2199-08-02 22:00:00,0,0,0,0
4,200003.0,2199-08-02 22:30:00,0,0,0,0


In [8]:
query = \
"""
SELECT * FROM ventdurations;
"""
vd = pd.read_sql_query(query, con)
#ventilation.icustay_id = ventilation.icustay_id.astype(int)
vd.head()

,icustay_id,ventnum,starttime,endtime,duration_hours
0,200003.0,1,2199-08-03 18:00:00,2199-08-07 13:00:00,91.000000
1,200006.0,1,2159-09-03 13:00:00,2159-09-04 03:50:00,14.833333
2,200009.0,1,2189-11-30 18:10:00,2189-12-01 10:00:00,15.833333
3,200011.0,1,2188-08-06 01:00:00,2188-08-06 04:00:00,3.000000
4,200014.0,1,2105-02-16 23:15:00,2105-02-17 18:00:00,18.750000


In [9]:
vt.noninvvent.sum()

178021

In [10]:
(vt.noninvvent * vt.mechvent).sum()

176843

In [11]:
query = \
"""
SELECT * FROM echo_filter_vars;
"""
efv = pd.read_sql_query(query, con)
#ventilation.icustay_id = ventilation.icustay_id.astype(int)
efv.head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter
0,83837,2182-08-19 10:46:00,264827,195409,32755,2182-08-18 16:50:19,2182-08-19 16:33:46,0 days 17:55:41,0 days 05:47:46,False,True,21643 days 16:50:19,NaN,NaN,NaT,NaT,True,True
1,104824,2119-01-26 14:49:00,276795,124493,69851,2119-01-26 16:57:32,2119-01-27 19:18:02,-1 days +21:51:28,1 days 04:29:02,False,True,25047 days 16:57:32,104888.0,104826.0,105 days 23:52:00,295 days 23:49:00,True,True
2,59659,2200-10-22 15:14:00,217810,188268,82208,2200-10-21 14:54:57,2200-10-24 14:01:58,1 days 00:19:03,1 days 22:47:58,False,True,22331 days 14:54:57,NaN,NaN,NaT,NaT,True,True
3,67634,2186-09-19 12:09:00,263932,168160,24182,2186-09-19 09:40:07,2186-09-22 13:15:57,0 days 02:28:53,3 days 01:06:57,True,True,27636 days 09:40:07,NaN,NaN,NaT,NaT,True,True
4,97569,2183-06-24 12:57:00,296461,107856,15977,2183-06-24 15:44:23,2183-06-28 16:12:29,-1 days +21:12:37,4 days 03:15:29,False,True,20237 days 15:44:23,97566.0,NaN,910 days 21:03:00,NaT,True,True


In [14]:
efv.sort_values('row_id').head()

,row_id,charttime,icustay_id,hadm_id,subject_id,intime,outtime,intime_to_echo,echo_to_outtime,ps_vaso,diag_xc,age_at_intime,after_rowid,before_rowid,op_to_icu,icu_to_op,time_filter,age_filter
7674,59653,2102-06-14 13:30:00,200492,174978,31038,2102-06-12 03:51:36,2102-06-14 14:40:25,2 days 09:38:24,0 days 01:10:25,False,True,21133 days 03:51:36,NaN,NaN,NaT,NaT,False,True
1413,59654,2191-05-10 15:08:00,245920,156140,70150,2191-05-09 15:18:03,2191-05-10 15:33:31,0 days 23:49:57,0 days 00:25:31,False,True,30260 days 15:18:03,NaN,NaN,NaT,NaT,True,True
22559,59655,2158-02-16 13:48:00,292743,188571,54190,2158-02-14 23:06:04,2158-02-21 01:24:38,1 days 14:41:56,4 days 11:36:38,False,False,31728 days 23:06:04,NaN,NaN,NaT,NaT,True,True
8779,59656,2173-09-02 09:47:00,274698,185291,5771,2173-09-01 21:23:50,2173-09-04 00:42:19,0 days 12:23:10,1 days 14:55:19,False,True,28541 days 21:23:50,NaN,NaN,NaT,NaT,True,True
7034,59657,2119-06-07 14:59:00,249621,100442,80030,2119-06-07 02:44:09,2119-06-14 18:38:33,0 days 12:14:51,7 days 03:39:33,True,True,20398 days 02:44:09,NaN,NaN,NaT,NaT,True,True


In [12]:
query = \
"""
SELECT * FROM ventfeatures;
"""
vf = pd.read_sql_query(query, con)
#ventilation.icustay_id = ventilation.icustay_id.astype(int)
vf.head()

,row_id,icustay_id,hadm_id,subject_id,noninv_vent,mech_vent
0,85251,272437,150204,82100,0,0
1,89924,292059,119420,82073,1,1
2,96614,223305,115762,19572,1,0
3,104632,211492,173690,60226,0,0
4,78926,266757,140374,841,1,0


In [15]:
vf.sort_values('row_id').head()

,row_id,icustay_id,hadm_id,subject_id,noninv_vent,mech_vent
19590,59653,200492,174978,31038,1,0
1945,59654,245920,156140,70150,0,0
8542,59655,292743,188571,54190,1,1
24024,59656,274698,185291,5771,0,0
20713,59657,249621,100442,80030,1,1


In [16]:
len(efv), len(vf)

(25512, 25512)